In [3]:
import os

In [4]:
print("man123")

man123


In [5]:
%pwd

'e:\\dsProject\\nycTaxiProject\\research'

In [6]:
os.chdir("../")

In [9]:
%pwd

'e:\\dsProject\\nycTaxiProject'

In [16]:
# ENTITY: DATA INGESTION
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    # unzip_dir: Path
    

In [17]:
from nycTaxiProject.constants import *
from nycTaxiProject.util.common import read_yaml, create_directories

In [18]:
class ConfigurationManager: # this class will read the config.yaml file and params.yaml file and create the directory for the artifacts
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):

            self.config=read_yaml(config_filepath)
            self.params=read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config=self.config.data_ingestion
          create_directories([config.root_dir]) # creating the directory for data ingestion
          
          data_ingestion_config=DataIngestionConfig(
                root_dir=config.root_dir,
                source_URL=config.source_URL,
                local_data_file=config.local_data_file
          )
          return data_ingestion_config

In [19]:
import kagglehub
import os
import zipfile
from nycTaxiProject import logger
from nycTaxiProject.util.common import get_size
import urllib.request as request

In [31]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config

    def download_file(self)->str:   # download the file from the URL to local directory
        """Fetch data from the URL to local directory"""
        try:
            dataset_url=self.config.source_URL
            download_path=self.config.local_data_file
            parent_dir=os.path.dirname(download_path) # why we are using parent dir because we want to create the directory if it does not exist before downloading the file
            os.makedirs(parent_dir,exist_ok=True)
            logger.info(f"downloading the file form{dataset_url} into file {download_path}")

            kagglehub.dataset_download(handle=dataset_url,output_dir=download_path,force_download=True)
            # filename,headers=request.urlretrieve(url=dataset_url,filename=download_path)

            logger.info(f"File downloaded successfully & saved @ {download_path} with size {get_size(Path(download_path))}")

        except Exception as e:
            raise e
        

# NO NEED FOR THIS FUNCTION CAUSE ALREADY IN A .CSV FORMAT

    # def extract_zip_file(self)->str: # extract the ZIP file into the unzip directory
    #     """Extract the ZIP file into the unzip directory"""
    #     try:
    #         zip_file_path=self.config.local_data_file # zip_file_path is the path to the ZIP file that needs to be extracted
    #         unzip_dir=self.config.unzip_dir # unzip_dir is the directory where the ZIP file will be extracted
    #         os.makedirs( unzip_dir,exist_ok=True)

    #         logger.info(f"Extacting the ZIP file {zip_file_path} into directory {unzip_dir}")
            
    #         with zipfile.ZipFile(zip_file_path,'r') as zip_ref:
    #             zip_ref.extractall(unzip_dir)
    #         logger.info(f"File extracted successfully in directory {unzip_dir} with size {get_size(unzip_dir)}")

    #     except Exception as e:
    #         raise e
        

In [32]:
try:
    config = ConfigurationManager()
    data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    # data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2026-02-20 12:43:41,219 : INFO : common : yaml file: config\config.yaml LOADED successfully]
[2026-02-20 12:43:41,226 : INFO : common : yaml file: params.yaml LOADED successfully]
[2026-02-20 12:43:41,230 : INFO : common : created directory at: artifacts]
[2026-02-20 12:43:41,236 : INFO : common : created directory at: artifacts/data_ingestion]
[2026-02-20 12:43:41,239 : INFO : 1430238311 : downloading the file formmann210cool/taxidata into file artifacts/data_ingestion]


100%|██████████| 130M/130M [00:23<00:00, 5.86MB/s] 

Extracting files...


[2026-02-20 12:44:19,028 : INFO : 1430238311 : File downloaded successfully & saved @ artifacts/data_ingestion with size ~ 0KB]


#### if file already there you don't need to delete it will automatically OVERWRITE that folder